# GBOModel Visualization

This notebook visualizes the GBOModel's input space.

## Setup

In [ ]:
import time, pathlib, sys

import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

from src.gbo.gbo_model import GBOModel
from src.utils import zero_mean_unit_var_normalization, zero_mean_unit_var_denormalization

In [ ]:
# Manual configuration
MODEL_CKPT   = "../results/debug_09/opt/iter_0/gbo_train_res.npz"
OPT_LATENTS  = "../results/debug_09/opt/iter_0/gbo_opt_res.npy"
OUT_PNG      = "gbomodel_tsne.png"
N_RANDOM     = 10_000

### Load model

In [ ]:
# Get checkpoint
ckpt = torch.load(MODEL_CKPT, map_location="cpu", weights_only=False)

X_mean = ckpt["X_mean"]
X_std  = ckpt["X_std"]

In [ ]:
model = GBOModel(
    input_dim   = ckpt["input_dim"],
    hidden_dims = ckpt["hidden_dims"],
    output_dim  = ckpt["output_dim"],
)
model.load_state_dict(ckpt["model_state_dict"])
model.eval()

d_latent = ckpt["input_dim"]
print(f"Loaded model  - latent dim: {d_latent}")

## Workflow

### Sample points

In [ ]:
torch.manual_seed(0)
Z_rand = torch.randn(N_RANDOM, d_latent)

Z_opt = np.load(OPT_LATENTS).squeeze()
Z_opt = zero_mean_unit_var_normalization(Z_opt, X_mean, X_std)
Z_opt = torch.tensor(Z_opt, dtype=torch.float32)

print(f"Random points    : {Z_rand.shape[0]}")
print(f"Optimized points : {Z_opt.shape[0]}")

### Evaluate model

In [ ]:
def batched_eval(model, Z, batch=2048):
    y_list = []
    with torch.inference_mode():
        for i in range(0, len(Z), batch):
            y_batch = model(Z[i:i+batch]).cpu()
            y_list.append(y_batch)
    return torch.cat(y_list, dim=0).numpy().ravel()

y_rand = batched_eval(model, Z_rand)
y_opt  = batched_eval(model, Z_opt)

### Denormalization

In [ ]:
X_rand = zero_mean_unit_var_denormalization(Z_rand, X_mean, X_std)
X_opt = zero_mean_unit_var_denormalization(Z_opt, X_mean, X_std)

### Print

In [ ]:
print(X_rand)
print(X_opt)

print(Z_rand)
print(Z_opt)

print(y_rand)
print(y_opt)

### t-SNE Projection

In [ ]:
X_all = torch.cat([X_rand, X_opt], dim=0).cpu().numpy()

print("Running t-SNE ... (this can take ~1-2 min on CPU)")
t0 = time.time()
tsne = TSNE(
    n_components=2,
    perplexity  = 50,          # good for 10 k–30 k points
    init        = "pca",
    learning_rate="auto",
    random_state=42,
)
X_2d = tsne.fit_transform(X_all)
print(f"t-SNE finished in {time.time() - t0:.1f}s")

X_rand_2d, X_opt_2d = X_2d[: len(X_rand)], X_2d[len(X_rand):]

In [ ]:
X_opt_2d

### Plot

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6))

sc = ax.scatter(
    X_rand_2d[:, 0], X_rand_2d[:, 1],
    c   = y_rand,
    s   = 6,
    alpha = 0.5,
    linewidths = 0,
)
ax.scatter(
    X_opt_2d[:, 0], X_opt_2d[:, 1],
    c   = y_opt,
    s   = 60,
    linewidths = 1.0,
    edgecolors = "k",
    marker = "o",
    label = "optimized",
)

cbar = plt.colorbar(sc, ax=ax)
cbar.set_label("model output")

ax.set_xlabel("t-SNE-1")
ax.set_ylabel("t-SNE-2")
ax.set_title("t-SNE projection of GBO latent space")
ax.legend()

plt.tight_layout()
plt.savefig(OUT_PNG, dpi=300)
print(f"Saved figure → {OUT_PNG}")
plt.show()